In [ ]:
import os
import re
from openai import OpenAI
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

client = OpenAI(api_key="OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

openai_embedding=OpenAIEmbeddings(model = 'text-embedding-3-small')

In [8]:
db3 = Chroma(
    persist_directory="./chroma_db",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
)

In [9]:
def translate_with_gpt(text, source_lang="ko", target_lang="en"):
    prompt = f"Translate this from {source_lang} to {target_lang}:\n\n{text}"

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )

    return response.choices[0].message.content.strip()


In [10]:
translate_with_gpt("문형빈 교수님의 논문 목록은")

"The list of Professor Mun Hyung-bin's papers is"

In [11]:
def classify_question_type(question_ko):
    prompt = f"""
다음 질문의 유형을 아래 중 하나로 분류해 주세요:
- 논문_목록
- 논문_요약
- 연구_흐름

질문: {question_ko}
질문 유형:"""
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt.strip()}],
        temperature=0
    )
    return response.choices[0].message.content.strip()

In [28]:
# 🔍 교수 이름 자동 추출 함수
def extract_professor_name(question: str) -> str | None:
    match = re.search(r"([가-힣]{2,4})\s*교수", question)
    return match.group(1) if match else None

# 📝 한글 질문 입력
question_ko = "노맹석 교수님의 논문 정리해줘"

# 1. 질문 유형 분류
question_type = classify_question_type(question_ko).strip()

# 2. 교수 이름 추출 (필요할 때만)
target_author = extract_professor_name(question_ko)
if question_type in ["논문_목록", "연구_흐름"] and not target_author:
    raise ValueError("질문에서 교수 이름을 추출할 수 없습니다.")

# 3. 영어 번역
question_en = translate_with_gpt(question_ko)

# 4. Chroma DB 검색
docs = db3.similarity_search(question_en, k=30)

# 5. 교수 이름 필터링 (필요할 경우만 적용)
if question_type in ["논문_목록", "연구_흐름"]:
    docs = [
        doc for doc in docs
        if target_author in doc.page_content or target_author in doc.metadata.get("author", "")
    ]

# 6. context 구성
context_text = "\n\n".join(doc.page_content for doc in docs)

# 7. 프롬프트 템플릿 정의
prompt_templates = {
    "논문_목록": PromptTemplate(
        input_variables=["context", "question"],
        template="""
You are provided with a collection of academic papers written by a professor. 
Based on the following user request, list the key papers along with:

1. The title of each paper  
2. The publication year (if available)  
3. A few core keywords representing the main topic  
4. The author(s) of each paper

User question:
{question}

Paper content:
{context}

📌 Please write your response in Korean using a respectful and organized tone.

논문 목록 요약 (in Korean):"""
    ),
    "논문_요약": PromptTemplate(
        input_variables=["context"],
        template="""
You are a research summarization assistant. Based on the following academic paper, provide a clear and concise summary including the following elements:

1. Research subject (what or who is being studied)  
2. Research method (how it was studied)  
3. Research findings (what was discovered)  
4. Suggestions or implications (recommendations or conclusions)

Paper content:
{context}

📌 Please write your summary in Korean, using a polite and professional tone.

논문 요약 (in Korean):"""
    ),
    "연구_흐름": PromptTemplate(
        input_variables=["context", "question"],
        template="""
You are an academic assistant. Given a collection of research papers written by a single professor, analyze how the research topics or areas of interest have evolved over time. 
Identify key shifts, trends, or patterns chronologically based on the publication content.

Context:
{context}

Question:
{question}

Now, summarize the chronological progression of the professor’s research focus. 
📌 Please write your answer in Korean using a clear and respectful tone.

연구 흐름 요약 (한국어로):"""
    )
}

# 8. 프롬프트 선택 + LLM 체인 구성
prompt = prompt_templates.get(question_type, prompt_templates["논문_요약"])
llm = ChatOpenAI(model="gpt-4o")
chain = prompt | llm

# 9. 실행
inputs = {"context": context_text}
if "question" in prompt.input_variables:
    inputs["question"] = question_ko

result = chain.invoke(inputs)

# 10. 출력
print(f"[{question_type.upper()}]")
print(result.content)


[논문_목록]
노맹석 교수님의 논문 목록을 정리하여 드리겠습니다. 아래는 주요 논문들의 정보입니다.

1. 논문 제목: **"스마트 팩토리에서의 인공지능 활용방안"**  
   출판 연도: 2020  
   핵심 키워드: 스마트 팩토리, 인공지능, 자동화  
   저자: 노맹석

2. 논문 제목: **"데이터 마이닝을 통한 소비자 행동 분석"**  
   출판 연도: 2018  
   핵심 키워드: 데이터 마이닝, 소비자 행동, 빅데이터  
   저자: 노맹석

3. 논문 제목: **"기계 학습을 이용한 금융 시장 예측"**  
   출판 연도: 2019  
   핵심 키워드: 기계 학습, 금융 시장, 예측 모델  
   저자: 노맹석, 김민수

4. 논문 제목: **"블록체인 기술의 보안성 분석"**  
   출판 연도: 2021  
   핵심 키워드: 블록체인, 보안, 암호화  
   저자: 노맹석, 이지은

5. 논문 제목: **"자율주행차의 센서 융합 기술"**  
   출판 연도: 2022  
   핵심 키워드: 자율주행차, 센서 융합, 기술 혁신  
   저자: 노맹석

위의 목록은 노맹석 교수님의 주요 연구 논문을 선별하여 정리한 것입니다. 추가적인 정보가 필요하시거나 다른 문의 사항이 있으시면 언제든지 말씀해 주세요.


In [ ]:
# 1. 한글 질문 입력
question_ko = "노맹석 교수님의 논문 목록"

# 2. 질문 유형 분류
question_type = classify_question_type(question_ko).strip()

# 3. 질문 영어로 번역 (Chroma DB 검색용)
question_en = translate_with_gpt(question_ko)

# 4. Chroma DB 검색
docs = db3.similarity_search(question_en, k=60)
filtered_docs = [doc for doc in docs if "문형빈" in doc.page_content or "문형빈" in doc.metadata.get("author", "")]
context_text = "\n\n".join([doc.page_content for doc in docs])

# 5. 프롬프트 템플릿 정의
prompt_templates = {
    "논문_목록": PromptTemplate(
        input_variables=["context", "question"],
        template="""
You are provided with a collection of academic papers written by a professor. 
Based on the following user request, list the key papers along with:

1. The title of each paper  
2. The publication year (if available)  
3. A few core keywords representing the main topic  
4. The author(s) of each paper

User question:
{question}

Paper content:
{context}

📌 Please write your response in Korean using a respectful and organized tone.

논문 목록 요약 (in Korean):"""
    ),
    "논문_요약": PromptTemplate(
        input_variables=["context"],
        template="""
You are a research summarization assistant. Based on the following academic paper, provide a clear and concise summary including the following elements:

1. Research subject (what or who is being studied)  
2. Research method (how it was studied)  
3. Research findings (what was discovered)  
4. Suggestions or implications (recommendations or conclusions)

Paper content:
{context}

📌 Please write your summary in Korean, using a polite and professional tone.

논문 요약 (in Korean):"""
    ),
    "연구_흐름": PromptTemplate(
        input_variables=["context", "question"],
        template="""
You are an academic assistant. Given a collection of research papers written by a single professor, analyze how the research topics or areas of interest have evolved over time. 
Identify key shifts, trends, or patterns chronologically based on the publication content.

Context:
{context}

Question:
{question}

Now, summarize the chronological progression of the professor’s research focus. 
📌 Please write your answer in Korean using a clear and respectful tone.

연구 흐름 요약 (한국어로):"""
    )
}

# 6. 프롬프트 선택 + LLM 구성
prompt = prompt_templates.get(question_type, prompt_templates["논문_요약"])
llm = ChatOpenAI(model="gpt-4o")
chain = prompt | llm  # 최신 LangChain 방식

# 7. 실행
inputs = {"context": context_text}
if "question" in prompt.input_variables:
    inputs["question"] = question_ko

result = chain.invoke(inputs)

# 8. 결과 출력
print(f"[{question_type.upper()}]")
print(result.content)

[논문_목록]
노맹석 교수님의 논문 목록 요약:

1. **논문 제목:** Hierarchical likelihood approach to non-Gaussian factor analysis  
   - **출판 연도:** 2019  
   - **핵심 키워드:** 계층적 가능도, 비정규 분포, 요인 분석  
   - **저자:** 노맹석, 이영조, Han Oud, Toni Toharudin  

2. **논문 제목:** Managing spent nuclear fuel in South Korea: Heterogeneous public attitudes toward different management strategies at individual- and segment levels  
   - **출판 연도:** 2020  
   - **핵심 키워드:** 사용후핵연료, 공공 태도, 관리 전략  
   - **저자:** 문형빈, 우종룰  

3. **논문 제목:** Robust estimation of dropout models using hierarchical likelihood  
   - **출판 연도:** 2010  
   - **핵심 키워드:** 드롭아웃 모델, 계층적 가능도, 강건 추정  
   - **저자:** 노맹석, 이영조, Michael G. Kenward  

4. **논문 제목:** Robust Ascertainment-Adjusted Parameter Estimation  
   - **출판 연도:** 정보 제공 안 됨  
   - **핵심 키워드:** 유전 연구, 강건 추정, 파라미터 추정  
   - **저자:** 노맹석, 이영조, Yudi Pawitan  

5. **논문 제목:** Attitudes in Korea toward Introducing Smart Policing Technologies: Differences between the General Public and Police Officers  
   - **출판 연도:*

In [ ]:
# 1. 한글 질문 입력
question_ko = "Managing spent nuclear fuel in South Korea: Heterogeneous public attitudes toward different management strategies at individual- and segment levels논문을 요약해줘"

# 2. 질문 유형 분류
question_type = classify_question_type(question_ko).strip()

# 3. 질문 영어로 번역 (Chroma DB 검색용)
question_en = translate_with_gpt(question_ko)

# 4. Chroma DB 검색
docs = db3.similarity_search(question_en, k=30)
context_text = "\n\n".join([doc.page_content for doc in docs])

# 5. 프롬프트 템플릿 정의
prompt_templates = {
    "논문_목록": PromptTemplate(
        input_variables=["context", "question"],
        template="""
You are provided with a collection of academic papers written by a professor. 
Based on the following user request, list the key papers along with:

1. The title of each paper  
2. The publication year (if available)  
3. A few core keywords representing the main topic

User question:
{question}

Paper content:
{context}

📌 Please write your response in Korean using a respectful and organized tone.

논문 목록 요약 (in Korean):"""
    ),
    "논문_요약": PromptTemplate(
        input_variables=["context"],
        template="""
You are a research summarization assistant. Based on the following academic paper, provide a clear and concise summary including the following elements:

1. Research subject (what or who is being studied)  
2. Research method (how it was studied)  
3. Research findings (what was discovered)  
4. Suggestions or implications (recommendations or conclusions)

Paper content:
{context}

📌 Please write your summary in Korean, using a polite and professional tone.

논문 요약 (in Korean):"""
    ),
    "연구_흐름": PromptTemplate(
        input_variables=["context", "question"],
        template="""
You are an academic assistant. Given a collection of research papers written by a single professor, analyze how the research topics or areas of interest have evolved over time. 
Identify key shifts, trends, or patterns chronologically based on the publication content.

Context:
{context}

Question:
{question}

Now, summarize the chronological progression of the professor’s research focus. 
📌 Please write your answer in Korean using a clear and respectful tone.

연구 흐름 요약 (한국어로):"""
    )
}

# 6. 프롬프트 선택 + LLM 구성
prompt = prompt_templates.get(question_type, prompt_templates["논문_요약"])
llm = ChatOpenAI(model="gpt-4o")
chain = prompt | llm  # 최신 LangChain 방식

# 7. 실행
inputs = {"context": context_text}
if "question" in prompt.input_variables:
    inputs["question"] = question_ko

result = chain.invoke(inputs)

# 8. 결과 출력
print(f"[{question_type.upper()}]")
print(result.content)

[논문_요약]
**연구대상**: 이 논문은 한국의 사용후 핵연료 관리에 대한 대중의 태도와 선호도를 조사했습니다. 연구는 개별 수준과 세그먼트 수준에서 대중의 선호를 분석했습니다.

**연구방법**: 연구는 선택 실험과 계층적 베이지안 정상 혼합 이산 선택 모델을 사용하여 수행되었습니다. 또한, 다양한 관리 전략에 대한 대중의 수용도를 사전 시뮬레이션을 통해 비교했습니다. 대중은 두 개의 세그먼트로 분류되었습니다: 세그먼트 1(31.62%)과 세그먼트 2(62.38%).

**연구결과**: 세그먼트 1은 전기 비용 증가에 더 민감하며 민주적 절차를 선호했습니다. 반면 세그먼트 2는 사용후 핵연료 재처리를 어느 정도 수용할 수 있었고, 영구 저장소에 대한 수용도가 높았습니다. 전체적으로 영구 저장소 건설 및 운영에 대한 대중의 수용도가 가장 높았습니다(55.00%).

**제언**: 한국 정부는 영구 저장소 건설 시 대중의 지지를 받을 가능성이 높으며, 세그먼트 1의 대중을 세그먼트 2로 이동시키기 위한 공공 정보 캠페인을 통해 사회적 합의를 도출할 필요가 있습니다. 또한, 정책 결정 과정에서 대중 참여를 강화하여 대중의 수용도를 높일 수 있습니다.


In [ ]:
# 1. 한글 질문 입력
question_ko = "노맹석 교수님의 연구 동향 알려줘"

# 2. 질문 유형 분류
question_type = classify_question_type(question_ko).strip()

# 3. 질문 영어로 번역 (Chroma DB 검색용)
question_en = translate_with_gpt(question_ko)

# 4. Chroma DB 검색
docs = db3.similarity_search(question_en, k=30)
context_text = "\n\n".join([doc.page_content for doc in docs])

# 5. 프롬프트 템플릿 정의
prompt_templates = {
    "논문_목록": PromptTemplate(
        input_variables=["context", "question"],
        template="""
You are provided with a collection of academic papers written by a professor. 
Based on the following user request, list the key papers along with:

1. The title of each paper  
2. The publication year (if available)  
3. A few core keywords representing the main topic

User question:
{question}

Paper content:
{context}

📌 Please write your response in Korean using a respectful and organized tone.

논문 목록 요약 (in Korean):"""
    ),
    "논문_요약": PromptTemplate(
        input_variables=["context"],
        template="""
You are a research summarization assistant. Based on the following academic paper, provide a clear and concise summary including the following elements:

1. Research subject (what or who is being studied)  
2. Research method (how it was studied)  
3. Research findings (what was discovered)  
4. Suggestions or implications (recommendations or conclusions)

Paper content:
{context}

📌 Please write your summary in Korean, using a polite and professional tone.

논문 요약 (in Korean):"""
    ),
    "연구_흐름": PromptTemplate(
        input_variables=["context", "question"],
        template="""
You are an academic assistant. Given a collection of research papers written by a single professor, analyze how the research topics or areas of interest have evolved over time. 
Identify key shifts, trends, or patterns chronologically based on the publication content.

Context:
{context}

Question:
{question}

Now, summarize the chronological progression of the professor’s research focus. 
📌 Please write your answer in Korean using a clear and respectful tone.

연구 흐름 요약 (한국어로):"""
    )
}

# 6. 프롬프트 선택 + LLM 구성
prompt = prompt_templates.get(question_type, prompt_templates["논문_요약"])
llm = ChatOpenAI(model="gpt-4o")
chain = prompt | llm  # 최신 LangChain 방식

# 7. 실행
inputs = {"context": context_text}
if "question" in prompt.input_variables:
    inputs["question"] = question_ko

result = chain.invoke(inputs)

# 8. 결과 출력
print(f"[{question_type.upper()}]")
print(result.content)

[연구_흐름]
노맹석 교수님의 연구는 다양한 분야에 걸쳐 발전해왔습니다. 초기에는 주로 통계 모델링과 관련된 연구를 진행하였고, 특히 비정규 분포를 다루는 계층적 일반화 선형 모델(HGLM) 및 분산 프레일티 모델에 대한 연구를 통해 의료 데이터 분석에서의 이질성을 설명하는 데 기여했습니다. 

이후, 그의 연구는 생물정보학과 의료/건강 정보학으로 확장되었습니다. 기계 학습과 데이터 마이닝 기법을 활용하여 질병과 miRNA의 연관성을 예측하는 연구를 수행함으로써, 생물의학적 문제 해결에 기여하고 있습니다. 이러한 연구들은 특히 생물정보학의 발전에 중요한 역할을 하고 있습니다.

최근에는 빅데이터 융합 분야에서의 연구를 통해 다양한 고차원 데이터의 분석 및 처리를 위한 새로운 방법론 개발에 집중하고 있습니다. 이러한 연구는 데이터 사이언스의 발전과 응용에 중요한 기여를 하고 있으며, 특히 의료 및 생물정보학 분야에서의 혁신적인 연구 결과를 도출하고 있습니다.

노맹석 교수님은 통계학, 생물정보학, 데이터 사이언스 등 다양한 분야에서 활발한 연구 활동을 통해 학문적 발전에 기여하고 있으며, 특히 데이터의 복잡성을 다루는 데 필요한 이론적 및 실용적 접근 방식을 개발하는 데 중점을 두고 있습니다.
